In [49]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

import torch.nn.functional as F

#plotting
import matplotlib.pyplot as plt

In [399]:
import data_utils
import models
import importlib
import transformer_models
importlib.reload(data_utils)
importlib.reload(models)
importlib.reload(transformer_models)
from data_utils import *
from models import *
from transformer_models import * 
import torch.optim as optim
from torch.utils.data import DataLoader
import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [400]:
data_dir = "data/"
dataObject = xrdData("data/", device)

In [401]:
dataObject.make_datasets(0.5, composition_embedding="composition1D")
# Create DataLoaders for train and validation sets
train_loader = DataLoader(dataObject.torch_datasets['train'], batch_size=256, shuffle=True)
valid_loader = DataLoader(dataObject.torch_datasets['val'], batch_size=256, shuffle=False)  

In [402]:
# Create the model instance and move it to the selected device
output_dim = 230  # Output dimension
ConvModel = XRD_C_SymNet(in_channels=1, output_dim=output_dim, composition_model= None).to(device)
token_size = 100 #dimension of the tokens 
TransModel = TransformerModel(ntoken = output_dim, d_model = token_size, nhead = 10, d_hid=100, nlayers=2, dropout = 0.3).to(device)
model = TransModel
# Define optimizer and loss function
weight_decay = 0  # Example value, adjust based on your needs
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [403]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [404]:
count_parameters(ConvModel)

31369900

In [405]:
count_parameters(TransModel)

993710

In [406]:
count_parameters(ConvModel) / count_parameters(TransModel)

31.56846564893178

In [407]:
#instantiate experimental data simulator 
simulator = ExperimentalSimulation(device) 

In [408]:
max_epochs = 200
metrics = ["accuracy", "loss"]

log = {
    f"{type}": {f"{metric}" : np.zeros(max_epochs) for metric in metrics} for type in ['train', 'val']     
}

for epoch in range(max_epochs):
    model.train()  # Set the model to training mode
    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for xrd, composition, targets in train_loader:
        if model == TransModel:
            xrd = tokenize_xrd(xrd, token_size=token_size)
        xrd = F.normalize(xrd, p=2, dim=1)
        optimizer.zero_grad()
        #xrd = simulator.sim(xrd)
        if model == TransModel:
            outputs = model(xrd) #add composition support later 
        else:
            outputs = model(xrd, composition)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()

    train_accuracy = 100 * correct_train / total_train
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_valid_loss = 0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():  # No gradients needed for validation
        for xrd, composition, targets in valid_loader:
            if model == TransModel:
                xrd = tokenize_xrd(xrd, token_size=token_size)
            xrd = F.normalize(xrd, p=2, dim=1)
            #xrd = simulator.sim(xrd)
            if model == TransModel:
                outputs = model(xrd) #add composition support later 
            else:
                outputs = model(xrd, composition)
            loss = criterion(outputs, targets)
            total_valid_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 
            total_valid += targets.size(0)
            correct_valid += (predicted == targets).sum().item()

    valid_accuracy = 100 * correct_valid / total_valid

    total_train_loss = total_train_loss / len(train_loader)
    validation_loss = total_valid_loss / len(valid_loader)

    print(f"Epoch {epoch+1}, Training Loss: {total_train_loss}, Training Accuracy: {train_accuracy}%, Validation Loss: {validation_loss}, Validation Accuracy: {valid_accuracy}%")

    log['train']['accuracy'][epoch] = (train_accuracy)
    log['train']['loss'][epoch] =(total_train_loss)

    log['val']['accuracy'][epoch] =(valid_accuracy)
    log['val']['loss'][epoch] =(validation_loss)

Epoch 1, Training Loss: 4.558626809590299, Training Accuracy: 8.759651169249569%, Validation Loss: 3.990792089038425, Validation Accuracy: 16.095511828432457%
Epoch 2, Training Loss: 3.8115406305017605, Training Accuracy: 17.963672721213133%, Validation Loss: 3.645903852250841, Validation Accuracy: 20.05306212690692%
Epoch 3, Training Loss: 3.541316052557717, Training Accuracy: 21.424207076598346%, Validation Loss: 3.431872248649597, Validation Accuracy: 22.46296705726288%
Epoch 4, Training Loss: 3.3610089765468114, Training Accuracy: 23.646058990168306%, Validation Loss: 3.313508907953898, Validation Accuracy: 24.121158523104135%
Epoch 5, Training Loss: 3.21701957810093, Training Accuracy: 26.034549797256012%, Validation Loss: 3.205666330125597, Validation Accuracy: 25.44771169577714%
Epoch 6, Training Loss: 3.1267404388374005, Training Accuracy: 27.001055379658947%, Validation Loss: 3.154497093624539, Validation Accuracy: 26.464735794826442%
Epoch 7, Training Loss: 3.047223772801144,

KeyboardInterrupt: 

In [204]:
model_dir = "models/"

date = datetime.now()
model_name = f"first_model_{date.month}_{date.day}"

os.makedirs(os.path.join(model_dir, model_name), exist_ok=True)

torch.save(model, os.path.join(model_dir, model_name, 'model.pth'))

for data_type, metrics_dict in log.items():
    for metric, array in metrics_dict.items():
        filename = f"{data_type}_{metric}.npy"  # Construct filename, e.g., "train_accuracy.npy"
        filename = os.path.join(model_dir, model_name, filename)
        np.save(filename, array)  # Save the array to a file

AttributeError: module 'datetime' has no attribute 'now'